In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import time

from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [5]:
import numpy as np
import pandas as pd

In [21]:
import os
os.mkdir("./Test2/")

In [19]:
pd.date_range('2014','2025',freq='YE')

DatetimeIndex(['2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
               '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31',
               '2022-12-31', '2023-12-31', '2024-12-31'],
              dtype='datetime64[ns]', freq='YE-DEC')

In [24]:
pd.date_range('2014','2025',freq='YS')

[Timestamp('2014-01-01 00:00:00'),
 Timestamp('2015-01-01 00:00:00'),
 Timestamp('2016-01-01 00:00:00'),
 Timestamp('2017-01-01 00:00:00'),
 Timestamp('2018-01-01 00:00:00'),
 Timestamp('2019-01-01 00:00:00'),
 Timestamp('2020-01-01 00:00:00'),
 Timestamp('2021-01-01 00:00:00'),
 Timestamp('2022-01-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2024-01-01 00:00:00'),
 Timestamp('2025-01-01 00:00:00')]

In [27]:
for date in pd.date_range('2014','2025',freq='YS'):
    print(str(date)[:-9])

2014-01-01
2015-01-01
2016-01-01
2017-01-01
2018-01-01
2019-01-01
2020-01-01
2021-01-01
2022-01-01
2023-01-01
2024-01-01
2025-01-01


In [2]:
search_term = "technological innovation"

In [4]:
sample_url = "https://arxiv.org/search/advanced"

In [2]:
key_word = "innovation"

In [148]:
class Article:
    def __init__(self):
        self.authors = ''
        self.title = ''
        self.abstract = ''
        self.submitted_date = ''
        self.pdf_link = ''

In [22]:
def send_key(driver,xpath,key_word):
    elm = driver.find_element(By.XPATH,xpath)
    elm.send_keys(key_word)

In [24]:
def set_search_term(driver,key_word,dropdown_value="abstract"):
    dropdown_xpath = '//*[@id="terms-0-field"]'
    dropdown = Select(driver.find_element(By.XPATH,dropdown_xpath))
    dropdown.select_by_value(dropdown_value)

    # input_box = driver.find_element(By.XPATH,input_box_xpath)
    # input_box.send_keys(key_word)
    send_key(driver,'//*[@id="terms-0-term"]',key_word)

In [17]:
def set_subject(driver):
    checkbox_xpath = '//*[@id="main-container"]/div[2]/div[2]/div[1]/div/form/section[2]/fieldset[1]/div[2]/div[1]/div[1]/div/div/label'
    checkbox = driver.find_element(By.XPATH,checkbox_xpath)
    checkbox.click()

In [29]:
def set_date_range(driver,date_from,date_to):
    label_xpath = '//*[@id="main-container"]/div[2]/div[2]/div[1]/div/form/section[2]/fieldset[2]/div[4]/div/div/label'
    label = driver.find_element(By.XPATH,label_xpath)
    label.click()
    time.sleep(0.5)

    date_from_xpath = '//*[@id="date-from_date"]'
    send_key(driver,date_from_xpath,date_from)
    
    date_to_xpath = '//*[@id="date-to_date"]'
    send_key(driver,date_to_xpath,date_to)

In [31]:
def click_search_button(driver):
    search_button_xpath = '//*[@id="main-container"]/div[2]/div[2]/div[1]/div/form/section[3]/button'
    search_button = driver.find_element(By.XPATH,search_button_xpath)
    search_button.click()

In [36]:
def click_next_button(driver):
    next_btn_xpath = '//*[@id="main-container"]/div[2]/nav[2]/a[2]'
    try:
        next_btn = driver.find_element(By.XPATH,next_btn_xpath)
        next_btn.click()
        return True
    except:
        return False

In [172]:
def extract_article_box(article_box):
    article = Article()
    try:
        more_btn = article_box.find_element(By.XPATH,'./p[3]/span[2]/a')
        more_btn.click()
        long_abstract = True
    except:
        long_abstract = False
    try:
        authors_xpath = './p[2]/a'
        authors = article_box.find_elements(By.XPATH,authors_xpath)
        author_list = []
        for author in authors:
            author_list.append(author.text)
        article.authors = ';'.join(author_list)
    except NoSuchElementException:
        print("Author tidak ditemukan")
        return article,False

    try:
        title_xpath = './p[1]'
        title = article_box.find_element(By.XPATH,title_xpath)
        article.title = title.text
    except NoSuchElementException:
        print("Title tidak ditemukan")
        return article,False

    try:
        if long_abstract:
            abstract = article_box.find_element(By.CSS_SELECTOR,'.abstract-full')
        else:
            abstract = article_box.find_element(By.CSS_SELECTOR,'.abstract-short')
        article.abstract = abstract.text
    except NoSuchElementException:
        print("Abstract tidak ditemukan")
        return article,False

    try:
        submitted_date = article_box.find_element(By.XPATH,'./p[4]')
        # pd.to_datetime(articles[0].submitted_date.)
        article.submitted_date = submitted_date.text.split(';')[0][10:]
    except NoSuchElementException:
        print("Submitted Date tidak ketemu")
        return article,False

    try:
        pdf_link = article_box.find_element(By.XPATH,'./div/p/span/a[1]')
        article.pdf_link = pdf_link.get_attribute('href')

        if 'pdf' not in article.pdf_link:
            raise('Link PDF tidak tersedia')
    except:
        print("Link PDF tidak tersedia")
        return article,False
        
    return article,True

In [173]:
def extract_article_boxes(driver):
    articles = []
    article_boxes_xpath = '//*[@id="main-container"]/div[2]/ol/li'
    article_boxes = driver.find_elements(By.XPATH,article_boxes_xpath)
    for article_box in article_boxes:
        article,valid = extract_article_box(article_box)
        if valid:
            articles.append(article)
    return articles

In [174]:
def get_data(driver,n=np.inf):
    articles = []
    i = 0
    while i < n:
        articles = articles + extract_article_boxes(driver)
        if not click_next_button(driver):
            break
        i += 1
        time.sleep(1)
    return articles

In [180]:
def article_list_to_df(article_list):
    df_dict = {
        'title': [],
        'authors':[],
        'abstract' : [],
        'submitted_date' : [],
        'pdf_link':[]
    }
    for article in article_list:
        df_dict['title'].append(article.title)
        df_dict['authors'].append(article.authors)
        df_dict['abstract'].append(article.abstract)
        df_dict['submitted_date'].append(article.submitted_date)
        df_dict['pdf_link'].append(article.pdf_link)
    return pd.DataFrame(df_dict)

In [184]:
driver = webdriver.Chrome()
driver.get(sample_url)
time.sleep(3)
set_search_term(driver,key_word)
set_subject(driver)
set_date_range(driver,date_from='2010-01-01',date_to='2025-07-18')
click_search_button(driver)
time.sleep(2)
articles = get_data(driver,n=1)

In [181]:
df = article_list_to_df(articles)

In [183]:
df.head()

,title,authors,abstract,submitted_date,pdf_link
0,A Survey on Segment Routing with Emphasis on U...,Aniruddha Kushwaha;Sidharth Sharma;Ashwin Gumaste,Segment routing is heralded as important techn...,"29 December, 2020",https://arxiv.org/pdf/2012.14687
1,Robotics Enabling the Workforce,Henrik Christensen;Maria Gini;Odest Chadwicke ...,Robotics has the potential to magnify the skil...,"16 December, 2020",https://arxiv.org/pdf/2012.09309
2,Use of Technology and Innovations in the COVID...,Adyasha Maharana;Morine Amutorine;Moinina Davi...,The use of technology has been ubiquitous in e...,"11 December, 2020",https://arxiv.org/pdf/2012.07741
3,Artificial Intelligence for COVID-19 Detection...,Parsa Sarosh;Shabir A. Parah;Romany F Mansur;G...,The emergence of COVID-19 has necessitated man...,"25 November, 2020",https://arxiv.org/pdf/2012.06310
4,Impact of Business technologies on the success...,Z. H. A Almtiri;S. J. Miah,The primary task of the study is to inspect th...,"11 December, 2020",https://arxiv.org/pdf/2012.06214
